In [1]:
import pypowsybl as pp
import pypowsybl.network as pn
import math

# Create network
network = pp.network.create_empty()

# Add substations
network.create_substations(
    id=['S1', 'S2', 'S3', 'S4'],
    name=['Hydro_Substation', 'Solar_Substation', 'Connection_Point', 'Grid_Substation'],
    country=['NO', 'NO', 'NO', 'NO'],
    tso=['TSO', 'TSO', 'TSO', 'TSO']
)

# Add voltage levels including 22kV for solar collection
network.create_voltage_levels(
    id=['VL1', 'VL2_22', 'VL2_132', 'VL3', 'VL4', 'VL5'],
    name=['Hydro_Generator', 'Solar_Plant_22kV', 'Solar_plant_substation_22/132kV', 'Connection_all plant_Alamoen_132KV', 'Reskjem Substation_132kV', 'Reskjem Substation_300kV'],
    substation_id=['S1', 'S2', 'S2', 'S3', 'S4', 'S4'],
    nominal_v=[132.0, 22.0, 132.0, 132.0, 132.0, 300.0],
    topology_kind=['BUS_BREAKER', 'BUS_BREAKER', 'BUS_BREAKER', 'BUS_BREAKER', 'BUS_BREAKER', 'BUS_BREAKER']
)

# Add buses
network.create_buses(
    id=['B1', 'B2_22', 'B2_132', 'B3', 'B4', 'B5'],
    name=['Hydro_Bus_PVbus', 'Solar_Collection_Bus', 'Solar_Trans_Bus', 'Connection_Bus', 'Reskjem Substation_132kV_Bus', 'Reskjem Substation_300kV_Bus'],
    voltage_level_id=['VL1', 'VL2_22', 'VL2_132', 'VL3', 'VL4', 'VL5']
)

# Add Hydro Generator (PV Bus)
network.create_generators(
    id=['G1'],
    voltage_level_id=['VL1'],
    bus_id=['B1'],
    target_p=[100.0],          # MW
    target_q=[0.0],            # MVAr
    target_v=[132.0],          # kV
    voltage_regulator_on=[True],
    min_p=[0.0],               # MW
    max_p=[100.0],             # MW
    rated_s=[120.0],           # MVA
    energy_source=['HYDRO']
)

# Add Solar collector feeders (7 x ~21.43MW at 22kV)
for i in range(7):
    network.create_generators(
        id=[f'Feeder_{i+1}'],
        voltage_level_id=['VL2_22'],
        bus_id=['B2_22'],
        target_p=[21.43],        # 150MW/5 feeders
        target_q=[0.0],
        target_v=[22.0],
        voltage_regulator_on=[True],
        min_p=[21.42],
        max_p=[22.0],
        rated_s=[30.0],
        energy_source=['OTHER']
    )


# Add Alamoen Generator (132kV)
network.create_generators(
    id=['G3'],
    name= ['Alamoen PV plant'],
    voltage_level_id=['VL3'],           # Connected to Alamoen 132kV bus
    bus_id=['B3'],
    target_p=[103.0],                   # 103 MW
    target_q=[0.0],
    target_v=[132.0],
    voltage_regulator_on=[True],
    min_p=[103.0],
    max_p=[103.0],
    rated_s=[120.0],                    # Slightly higher than power output
    energy_source=['OTHER']
)


# Add Grid Slack Bus with reversed power convention
network.create_generators(
    id=['GRID_SLACK'],
    voltage_level_id=['VL5'],
    bus_id=['B5'],
    target_p=[0.0],          # Initial target
    target_q=[0.0],
    target_v=[300.0],
    voltage_regulator_on=[True],
    min_p=[-1000.0],         # Allow large negative power (generation)
    max_p=[1000.0],          # Allow large positive power (absorption)
    rated_s=[1000.0],
    energy_source=['OTHER']
)

# Add load at slack bus
network.create_loads(
    id=['L1'],
    name=['Grid_Load'],
    voltage_level_id=['VL5'],
    bus_id=['B5'],    # Updated to match actual bus ID
    p0=[353.0],
    q0=[0.0]
)

# Calculate line parameters
def calculate_line_parameters(length_km):
    r_per_km = 0.071145  # ohm/km at 20°C
    x_per_km = 0.394    # ohm/km at 50Hz
    c_per_km = 11.1e-9  # F/km
    f = 50  # Hz
    
    r_total = r_per_km * length_km
    x_total = x_per_km * length_km
    b_total = 2 * math.pi * f * c_per_km * length_km
    
    return {
        'r': r_total,
        'x': x_total,
        'b': b_total
    }

# Calculate parameters for each section
hydro_params = calculate_line_parameters(10)    # 10km for hydro
solar_params = calculate_line_parameters(1.8)   # 1.8km for solar
grid_params = calculate_line_parameters(15)     # 15km for grid connection

# Add Hydro plant lines (2 x 10km)
network.create_lines(
    id=['L1_H1', 'L1_H2'],
    name=['Hydro_Line_1 (10km)', 'Hydro_Line_2 (10km)'],
    voltage_level1_id=['VL1', 'VL1'],
    bus1_id=['B1', 'B1'],
    voltage_level2_id=['VL3', 'VL3'],
    bus2_id=['B3', 'B3'],
    b1=[hydro_params['b'], hydro_params['b']],
    b2=[hydro_params['b'], hydro_params['b']],
    g1=[0.0, 0.0],
    g2=[0.0, 0.0],
    r=[hydro_params['r'], hydro_params['r']],
    x=[hydro_params['x'], hydro_params['x']],
)

# Add solar plant transformer (22/132 kV)
network.create_2_windings_transformers(
    id=['TR1_A', 'TR1_B'],
    name=['Solar_Transformer_A', 'Solar_Transformer_B'],
    voltage_level1_id=['VL2_22', 'VL2_22'],
    voltage_level2_id=['VL2_132', 'VL2_132'],
    bus1_id=['B2_22', 'B2_22'],
    bus2_id=['B2_132', 'B2_132'],
    rated_u1=[22.0, 22.0],               # Primary voltage (kV)
    rated_u2=[132.0, 132.0],             # Secondary voltage (kV)
    rated_s=[100.0, 100.0],              # Rated power (MVA)
    r=[0.725, 0.725],                    # Resistance referred to HV side (Ω)
    x=[21.75, 21.75],                    # Reactance referred to HV side (Ω)
    g=[5.7e-6, 5.7e-6],                  # Conductance from no-load losses (S)
    b=[9.92e-6, 9.92e-6]                 # Susceptance from magnetizing current (S)
)

# Add transmission line from solar plant (1.8km at 132kV)
network.create_lines(
    id=['L2_S1'],
    name=['Solar_Line (1.8km, 150MW)'],
    voltage_level1_id=['VL2_132'],
    voltage_level2_id=['VL3'],
    bus1_id=['B2_132'],
    bus2_id=['B3'],
    r=[solar_params['r']],
    x=[solar_params['x']],
    g1=[0.0],
    b1=[solar_params['b']],
    g2=[0.0],
    b2=[solar_params['b']]
)

# Add Grid connection lines (2 x 15km)
network.create_lines(
    id=['L3_G1', 'L3_G2'],
    name=['Grid_Line_1 (15km)', 'Grid_Line_2 (15km)'],
    voltage_level1_id=['VL3', 'VL3'],
    voltage_level2_id=['VL4', 'VL4'],
    bus1_id=['B3', 'B3'],
    bus2_id=['B4', 'B4'],
    r=[grid_params['r'], grid_params['r']],
    x=[grid_params['x'], grid_params['x']],
    g1=[0.0, 0.0],
    b1=[grid_params['b'], grid_params['b']],
    g2=[0.0, 0.0],
    b2=[grid_params['b'], grid_params['b']]
)

# Add main grid transformers (3 × 160MVA, 132/300 kV)
network.create_2_windings_transformers(
    id=['T1_A', 'T1_B', 'T1_C'],
    name=['Grid_Transformer_A', 'Grid_Transformer_B', 'Grid_Transformer_C'],
    voltage_level1_id=['VL4', 'VL4', 'VL4'],
    voltage_level2_id=['VL5', 'VL5', 'VL5'],
    bus1_id=['B4', 'B4', 'B4'],
    bus2_id=['B5', 'B5', 'B5'],
    rated_u1=[132.0, 132.0, 132.0],       # Primary voltage (kV)
    rated_u2=[300.0, 300.0, 300.0],       # Secondary voltage (kV)
    rated_s=[160.0, 160.0, 160.0],        # Rated power (MVA)
    r=[1.76, 1.76, 1.76],                 # Resistance referred to HV side (Ω)
    x=[70.4, 70.4, 70.4],                 # Reactance referred to HV side (Ω)
    g=[1.78e-6, 1.78e-6, 1.78e-6],       # Conductance (S)
    b=[3.08e-6, 3.08e-6, 3.08e-6]        # Susceptance (S)
)

# Run power flow with parameters
loadflow_parameters = pp.loadflow.Parameters(
    voltage_init_mode=pp.loadflow.VoltageInitMode.UNIFORM_VALUES,
    transformer_voltage_control_on=True,
    use_reactive_limits=False,
    distributed_slack=False,
    balance_type=pp.loadflow.BalanceType.PROPORTIONAL_TO_GENERATION_P,
    write_slack_bus=True,    # Add this to identify selected slack bus
    read_slack_bus=True      # Add this to use defined slack bus
)

# Run the power flow analysis with these parameters
results = pp.loadflow.run_ac(network, parameters=loadflow_parameters)


print("\nConvergence Status:")
for result in results:
    print(f"Status: {result.status}")
    print(f"Details: {result.status_text}")

print("\n=== System Configuration ===")
print("1. Solar Plant (22kV Collection):")
print("- 5 feeders x 21.43 MW = 150 MW")
print("- Step-up transformer: 22/132 kV, 100 MVA")
print("- Transmission line: 1.8 km at 132kV")

print("\n2. Hydro Plant:")
print("- Generation: 100 MW")
print("- Voltage: 132 kV")
print("- Transmission: 2 x 10 km lines")

print("\n3. Grid Connection:")
print("- 2 x 15 km lines at 132 kV")
print("- Main transformer: 132/300 kV, 400 MVA")

#Slack generator
gen_results = network.get_generators()
slack_gen = gen_results.loc['GRID_SLACK']
print("\n=== Slack Bus Analysis ===")
print(f"Location: {slack_gen['name']}")
print(f"Voltage: {slack_gen['target_v']:.2f} kV")
print(f"Power absorbed: {abs(slack_gen['p']):.2f} MW")      # Changed from target_p to p
print(f"Reactive power: {slack_gen['q']:.2f} MVAR")         # Changed from target_q to q
print("\nTarget values:")
print(f"Target P: {slack_gen['target_p']:.2f} MW")
print(f"Target Q: {slack_gen['target_q']:.2f} MVAR")

# Get bus voltages and calculate voltage drops
bus_results = network.get_buses()
print("\n=== Voltage Profile and Drops ===")
nominal_voltages = {
    'VL1': 132.0,     # Hydro
    'VL2_22': 22.0,   # Solar collection
    'VL2_132': 132.0, # Solar transmission
    'VL3': 132.0,     # Connection Point
    'VL4': 132.0,     # Grid LV
    'VL5': 300.0      # Grid HV
}

for bus_id in bus_results.index:
    v_mag = bus_results.loc[bus_id, 'v_mag']
    v_angle = bus_results.loc[bus_id, 'v_angle']
    vl_id = bus_results.loc[bus_id, 'voltage_level_id']
    nominal_v = nominal_voltages[vl_id]
    voltage_deviation = ((v_mag - nominal_v)/nominal_v) * 100
    
    print(f"\nBus: {bus_results.loc[bus_id, 'name']}")
    print(f"Actual Voltage: {v_mag:.2f} kV")
    print(f"Nominal Voltage: {nominal_v:.2f} kV")
    print(f"Voltage Deviation: {voltage_deviation:+.2f}%")
    print(f"Angle: {v_angle:.4f}°")

# Analyze transmission path voltage drops
print("\n=== Transmission Path Voltage Drop Analysis ===")

print("\n1. Solar Collection System (22kV):")
# Analyze voltage drops at solar collection level
solar_collection_voltage = bus_results.loc[bus_results['voltage_level_id'] == 'VL2_22', 'v_mag'].values[0]
print(f"Collection Bus Voltage: {solar_collection_voltage:.2f} kV")

print("\n2. Solar Plant Transformer (22/132 kV):")
transformer_results = network.get_2_windings_transformers()
# Analyze both transformers
for transformer_id in ['TR1_A', 'TR1_B']:
    solar_transformer = transformer_results.loc[transformer_id]
    print(f"\nTransformer {transformer_id}:")
    print(f"LV Side (22kV): {solar_collection_voltage:.2f} kV")
    print(f"HV Side (132kV): {bus_results.loc[bus_results['voltage_level_id'] == 'VL2_132', 'v_mag'].values[0]:.2f} kV")
    
    # Calculate loading
    p1 = solar_transformer['p1']
    p2 = solar_transformer['p2']
    losses = abs(p1 + p2)
    loading = (abs(p1)/float(solar_transformer['rated_s'])) * 100
    print(f"Power Flow: {abs(p1):.2f} MW")
    print(f"Losses: {losses:.2f} MW")
    print(f"Loading: {loading:.1f}%")

print("\n3. Solar Transmission Line (1.8km):")
solar_line = network.get_lines().loc['L2_S1']
sending_v = bus_results.loc[solar_line['bus1_id'], 'v_mag']
receiving_v = bus_results.loc[solar_line['bus2_id'], 'v_mag']
voltage_drop = sending_v - receiving_v
print(f"Sending End: {sending_v:.2f} kV")
print(f"Receiving End: {receiving_v:.2f} kV")
print(f"Voltage Drop: {voltage_drop:.2f} kV ({(voltage_drop/sending_v*100):+.2f}%)")

print("\n4. Hydro Plant Lines (2 x 10km):")
for line_id in ['L1_H1', 'L1_H2']:
    line_results = network.get_lines()
    sending_bus = line_results.loc[line_id, 'bus1_id']
    receiving_bus = line_results.loc[line_id, 'bus2_id']
    v1 = bus_results.loc[sending_bus, 'v_mag']
    v2 = bus_results.loc[receiving_bus, 'v_mag']
    voltage_drop = v1 - v2
    print(f"\nLine {line_id}:")
    print(f"Sending End: {v1:.2f} kV")
    print(f"Receiving End: {v2:.2f} kV")
    print(f"Voltage Drop: {voltage_drop:.2f} kV ({(voltage_drop/v1*100):+.2f}%)")

print("\n5. Grid Connection Lines (2 x 15km):")
for line_id in ['L3_G1', 'L3_G2']:
    line_results = network.get_lines()
    sending_bus = line_results.loc[line_id, 'bus1_id']
    receiving_bus = line_results.loc[line_id, 'bus2_id']
    v1 = bus_results.loc[sending_bus, 'v_mag']
    v2 = bus_results.loc[receiving_bus, 'v_mag']
    voltage_drop = v1 - v2
    print(f"\nLine {line_id}:")
    print(f"Sending End: {v1:.2f} kV")
    print(f"Receiving End: {v2:.2f} kV")
    print(f"Voltage Drop: {voltage_drop:.2f} kV ({(voltage_drop/v1*100):+.2f}%)")

print("\n6. Main Transformers (132/300 kV):")
transformer_results = network.get_2_windings_transformers()

# Analyze each main transformer
for transformer_id in ['T1_A', 'T1_B', 'T1_C']:
    main_transformer = transformer_results.loc[transformer_id]
    lv_voltage = bus_results.loc[main_transformer['bus1_id'], 'v_mag']
    hv_voltage = bus_results.loc[main_transformer['bus2_id'], 'v_mag']
    
    print(f"\nTransformer {transformer_id}:")
    print(f"LV Side: {lv_voltage:.2f} kV")
    print(f"HV Side: {hv_voltage:.2f} kV")
    print(f"Transformation Ratio: {(hv_voltage/lv_voltage):.4f}")
    
    # Calculate loading and losses
    p1 = main_transformer['p1']
    p2 = main_transformer['p2']
    losses = abs(p1 + p2)
    loading = (abs(p1)/float(main_transformer['rated_s'])) * 100
    print(f"Loading: {loading:.1f}%")
    print(f"Losses: {losses:.2f} MW")

# Calculate total transformer stats
total_main_transformer_losses = sum(abs(transformer_results.loc[tid, 'p1'] + 
                                      transformer_results.loc[tid, 'p2']) 
                                  for tid in ['T1_A', 'T1_B', 'T1_C'])

print(f"\nTotal Main Transformer System:")
print(f"Total Losses: {total_main_transformer_losses:.2f} MW")
average_loading = sum(abs(transformer_results.loc[tid, 'p1'])/
                     float(transformer_results.loc[tid, 'rated_s']) * 100 
                     for tid in ['T1_A', 'T1_B', 'T1_C'])/3
print(f"Average Loading: {average_loading:.1f}%")

# Get line flows
line_results = network.get_lines()
print("\nLine Power Flows:")
for line_id in line_results.index:
    print(f"\nLine {line_results.loc[line_id, 'name']}:")
    p1 = line_results.loc[line_id, 'p1']
    q1 = line_results.loc[line_id, 'q1']
    p2 = line_results.loc[line_id, 'p2']
    q2 = line_results.loc[line_id, 'q2']
    i1 = line_results.loc[line_id, 'i1']
    losses = abs(p1 + p2)
    print(f"Power Flow: {p1:.2f} MW + j{q1:.2f} MVAr → {-p2:.2f} MW + j{-q2:.2f} MVAr")
    print(f"Current: {i1:.2f} A")
    print(f"Losses: {losses:.2f} MW")

# Create SLD
# Create base SLD parameters
param = pn.SldParameters(
    use_name=True,
    center_name=True,
    diagonal_label=False,
    nodes_infos=True,
    topological_coloring=True,
    component_library="FlatDesign",
    active_power_unit="MW",
    reactive_power_unit="MVAR",
    display_current_feeder_info=True
)

# Layout 1: 2x2 Matrix
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2'], 
     ['S3', 'S4']], 
    'sld_2x2_matrix_solarplant_22KV.svg',
    parameters=param
)

# Layout 2: Vertical Flow
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1'], 
     ['S2'],
     ['S3'],
     ['S4']], 
    'sld_vertical_solarplant_22KV.svg',
    parameters=param
)

# Layout 3: Horizontal Flow
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2', 'S3', 'S4']], 
    'sld_horizontal_solarplant_22KV.svg',
    parameters=param
)

# Layout 4: L-shaped
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2', 'S3'], 
     ['', '', 'S4']], 
    'sld_L_shape_solarplant_22KV.svg',
    parameters=param
)

# Layout 6: Separated Generation and Grid
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2', ''], 
     ['', 'S3', 'S4']], 
    'sld_separated_solarplant_22KV.svg',
    parameters=param
)

# Layout 7: Cascade
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1'], 
     ['S2', 'S3'], 
     ['', 'S4']], 
    'sld_cascade_solarplant_22KV.svg',
    parameters=param
)

# Also try different component libraries
param_flat = pn.SldParameters(
    use_name=True,
    center_name=True,
    diagonal_label=False,
    nodes_infos=True,
    topological_coloring=True,
    component_library="FlatDesign",
    active_power_unit= "mw",
    reactive_power_unit="MVAR",
    display_current_feeder_info=True
)

param_convergence = pn.SldParameters(
    use_name=True,
    center_name=True,
    diagonal_label=False,
    nodes_infos=True,
    topological_coloring=True,
    component_library="Convergence",
    active_power_unit="MW",
    reactive_power_unit="MVAR",
    display_current_feeder_info=True
)

# Save with different component libraries
network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2'], ['S3', 'S4']], 
    'sld_flat_design_solarplant_22KV.svg',
    parameters=param_flat
)

network.write_matrix_multi_substation_single_line_diagram_svg(
    [['S1', 'S2'], ['S3', 'S4']], 
    'sld_convergence_solarplant_22KV.svg',
    parameters=param_convergence
)

print("Generated multiple SLD layouts:")



Convergence Status:
Status: ComponentStatus.CONVERGED
Details: Converged

=== System Configuration ===
1. Solar Plant (22kV Collection):
- 5 feeders x 21.43 MW = 150 MW
- Step-up transformer: 22/132 kV, 100 MVA
- Transmission line: 1.8 km at 132kV

2. Hydro Plant:
- Generation: 100 MW
- Voltage: 132 kV
- Transmission: 2 x 10 km lines

3. Grid Connection:
- 2 x 15 km lines at 132 kV
- Main transformer: 132/300 kV, 400 MVA

=== Slack Bus Analysis ===
Location: 
Voltage: 300.00 kV
Power absorbed: 0.00 MW
Reactive power: -55.72 MVAR

Target values:
Target P: 0.00 MW
Target Q: 0.00 MVAR

=== Voltage Profile and Drops ===

Bus: Hydro_Generator_0
Actual Voltage: 132.00 kV
Nominal Voltage: 132.00 kV
Voltage Deviation: +0.00%
Angle: 9.3805°

Bus: Solar_Plant_22kV_0
Actual Voltage: 22.00 kV
Nominal Voltage: 22.00 kV
Voltage Deviation: +0.00%
Angle: 14.4266°

Bus: Solar_plant_substation_22/132kV_0
Actual Voltage: 132.07 kV
Nominal Voltage: 132.00 kV
Voltage Deviation: +0.06%
Angle: 9.0657°

Bus:

In [2]:
# After power flow runs, add this detailed analysis
print("\n=== Power Balance Analysis ===")

# Get all generations
gen_results = network.get_generators()
total_generation = 0
print("\nGeneration:")
for gen_id in gen_results.index:
    if gen_id != 'GRID_SLACK':
        power = abs(gen_results.loc[gen_id, 'target_p'])
        total_generation += power
        print(f"{gen_id}: {power:.2f} MW")
print(f"Total Generation: {total_generation:.2f} MW")

# Get all loads
load_results = network.get_loads()
total_load = 0
print("\nLoads:")
for load_id in load_results.index:
    power = load_results.loc[load_id, 'p0']
    total_load += power
    print(f"{load_id}: {power:.2f} MW")
print(f"Total Load: {total_load:.2f} MW")

# Get slack bus details
slack_gen = gen_results.loc['GRID_SLACK']
print("\nSlack Bus Details:")
print(f"Power: {slack_gen['p']:.2f} MW")  # Actual power, not target
print(f"Target Power: {slack_gen['target_p']:.2f} MW")
print(f"Min Power: {slack_gen['min_p']:.2f} MW")
print(f"Max Power: {slack_gen['max_p']:.2f} MW")
print(f"Bus ID: {slack_gen['bus_id']}")

# Check load connection
grid_load = load_results.loc['L1']
print("\nGrid Load Details:")
print(f"Bus ID: {grid_load['bus_id']}")
print(f"Power: {grid_load['p0']:.2f} MW")
print(f"Connected to voltage level: {grid_load['voltage_level_id']}")




=== Power Balance Analysis ===

Generation:
G1: 100.00 MW
Feeder_1: 21.43 MW
Feeder_2: 21.43 MW
Feeder_3: 21.43 MW
Feeder_4: 21.43 MW
Feeder_5: 21.43 MW
Feeder_6: 21.43 MW
Feeder_7: 21.43 MW
G3: 103.00 MW
Total Generation: 353.01 MW

Loads:
L1: 353.00 MW
Total Load: 353.00 MW

Slack Bus Details:
Power: -0.00 MW
Target Power: 0.00 MW
Min Power: -1000.00 MW
Max Power: 1000.00 MW
Bus ID: VL5_0

Grid Load Details:
Bus ID: VL5_0
Power: 353.00 MW
Connected to voltage level: VL5


In [3]:
# Detailed Power Flow Analysis
print("\n=== Detailed Power Flow Analysis ===")

# Check all generators
print("\nGenerator Power Output:")
for gen_id in gen_results.index:
    gen = gen_results.loc[gen_id]
    print(f"{gen_id}:")
    print(f"  Actual P: {gen['p']:.2f} MW")
    print(f"  Target P: {gen['target_p']:.2f} MW")
    print(f"  Actual Q: {gen['q']:.2f} MVAR")
    print(f"  Bus ID: {gen['bus_id']}")

# Check all loads
load_results = network.get_loads()
print("\nLoad Power Consumption:")
for load_id in load_results.index:
    load = load_results.loc[load_id]
    print(f"{load_id}:")
    print(f"  Actual P: {load['p0']:.2f} MW")
    print(f"  Bus ID: {load['bus_id']}")

# Check power flow parameters
print("\nPower Flow Parameters:")
print(f"Distributed Slack: {loadflow_parameters.distributed_slack}")
print(f"Balance Type: {loadflow_parameters.balance_type}")


=== Detailed Power Flow Analysis ===

Generator Power Output:
G1:
  Actual P: -100.00 MW
  Target P: 100.00 MW
  Actual Q: 18.67 MVAR
  Bus ID: VL1_0
Feeder_1:
  Actual P: -21.43 MW
  Target P: 21.43 MW
  Actual Q: -0.11 MVAR
  Bus ID: VL2_22_0
Feeder_2:
  Actual P: -21.43 MW
  Target P: 21.43 MW
  Actual Q: -0.11 MVAR
  Bus ID: VL2_22_0
Feeder_3:
  Actual P: -21.43 MW
  Target P: 21.43 MW
  Actual Q: -0.11 MVAR
  Bus ID: VL2_22_0
Feeder_4:
  Actual P: -21.43 MW
  Target P: 21.43 MW
  Actual Q: -0.11 MVAR
  Bus ID: VL2_22_0
Feeder_5:
  Actual P: -21.43 MW
  Target P: 21.43 MW
  Actual Q: -0.11 MVAR
  Bus ID: VL2_22_0
Feeder_6:
  Actual P: -21.43 MW
  Target P: 21.43 MW
  Actual Q: -0.11 MVAR
  Bus ID: VL2_22_0
Feeder_7:
  Actual P: -21.43 MW
  Target P: 21.43 MW
  Actual Q: -0.11 MVAR
  Bus ID: VL2_22_0
G3:
  Actual P: -103.00 MW
  Target P: 103.00 MW
  Actual Q: -24.03 MVAR
  Bus ID: VL3_0
GRID_SLACK:
  Actual P: -0.00 MW
  Target P: 0.00 MW
  Actual Q: -55.72 MVAR
  Bus ID: VL5_0

L

In [4]:
# Loss Analysis
print("\n=== Loss Analysis for Complete System ===")

# Calculate total generation
total_generation = 0
gen_results = network.get_generators()
print("\nGeneration Summary:")
for gen_id in gen_results.index:
    power = abs(gen_results.loc[gen_id, 'target_p'])
    total_generation += power
    print(f"Generator {gen_id}: {power:.2f} MW")
print(f"Total Generation: {total_generation:.2f} MW")

# 22kV System Analysis
print("\n22kV System Losses:")
ring_losses = 0
for line_id in ['L2_S1']:  # Solar line
    line = network.get_lines().loc[line_id]
    line_loss = abs(line['p1'] + line['p2'])
    ring_losses += line_loss
    current = line['i1']
    print(f"\n{line['name']} (22kV):")
    print(f"Current: {current:.2f} A")
    print(f"Losses: {line_loss:.2f} MW")
    print(f"Loss percentage: {(line_loss/abs(line['p1'])*100):.2f}%")

# Calculate 132kV system losses
line_results = network.get_lines()

# Hydro lines losses
hydro_losses = 0
for line_id in ['L1_H1', 'L1_H2']:  # Hydro lines
    line = line_results.loc[line_id]
    hydro_losses += abs(line['p1'] + line['p2'])
    print(f"\nHydro Line {line['name']}:")
    print(f"Current: {line['i1']:.2f} A")
    print(f"Losses: {abs(line['p1'] + line['p2']):.2f} MW")

# Grid lines losses
grid_losses = 0
for line_id in ['L3_G1', 'L3_G2']:  # Grid lines
    line = line_results.loc[line_id]
    grid_losses += abs(line['p1'] + line['p2'])
    print(f"\nGrid Line {line['name']}:")
    print(f"Current: {line['i1']:.2f} A")
    print(f"Losses: {abs(line['p1'] + line['p2']):.2f} MW")

# Total 132kV losses including Alamoen generation impact
total_132kv_losses = hydro_losses + grid_losses

# Solar transformer losses (22/132 kV)
transformer_losses = 0
transformer_results = network.get_2_windings_transformers()
for trans_id in ['TR1_A', 'TR1_B']:
    transformer = transformer_results.loc[trans_id]
    transformer_loss = abs(transformer['p1'] + transformer['p2'])
    transformer_losses += transformer_loss
    loading = (abs(transformer['p1'])/float(transformer['rated_s'])) * 100
    print(f"\n22/132kV {transformer['name']}:")
    print(f"Power Flow: {abs(transformer['p1']):.2f} MW")
    print(f"Losses: {transformer_loss:.2f} MW")
    print(f"Loss percentage: {(transformer_loss/abs(transformer['p1'])*100):.2f}%")
    print(f"Loading: {loading:.1f}%")

# Main transformers losses (132/300 kV)
print("\nMain Transformers Analysis (Including Alamoen Power Flow):")
main_transformer_losses = 0
for trans_id in ['T1_A', 'T1_B', 'T1_C']:
    transformer = transformer_results.loc[trans_id]
    transformer_loss = abs(transformer['p1'] + transformer['p2'])
    main_transformer_losses += transformer_loss
    loading = (abs(transformer['p1'])/float(transformer['rated_s'])) * 100
    print(f"\n132/300kV {transformer['name']}:")
    print(f"Power Flow: {abs(transformer['p1']):.2f} MW")
    print(f"Losses: {transformer_loss:.2f} MW")
    print(f"Loss percentage: {(transformer_loss/abs(transformer['p1'])*100):.2f}%")
    print(f"Loading: {loading:.1f}%")

# Cable specifications
FEEDER_LENGTH = 0.5  # km (500 meters)
R_DC_20 = 0.0786    # Ω/km at 20°C for 500mm² cable
CURRENT_RATING = 650 # A
# Temperature correction for resistance (90°C operating temp)
R_90 = 0.0786

# Feeder losses (5 feeders at 22kV)
feeder_losses = 0
total_feeder_current = 0

for i in range(7):
    gen_id = f'G2_{i+1}'
    # Calculate current
    feeder_power = 21.43  # MW (150MW/5)
    feeder_current = (feeder_power * 1e6) / (math.sqrt(3) * 22000)  # A
    total_feeder_current += feeder_current
    
    # Calculate losses
    r_total = R_90 * FEEDER_LENGTH  # Total resistance for feeder length
    feeder_loss = 3 * (feeder_current ** 2 * r_total) / 1e6  # MW (3-phase)
    feeder_losses += feeder_loss
    
    loading_percentage = (feeder_current / CURRENT_RATING) * 100
    
    print(f"\nFeeder {i+1}:")
    print(f"Power: {feeder_power:.2f} MW")
    print(f"Current: {feeder_current:.2f} A")
    print(f"Loading: {loading_percentage:.1f}% of rating")
    print(f"Losses: {feeder_loss:.3f} MW")
    print(f"Loss percentage: {(feeder_loss/feeder_power*100):.3f}%")

print(f"\nTotal Feeder Statistics:")
print(f"Total Feeder Losses: {feeder_losses:.3f} MW")
print(f"Average Loading: {(total_feeder_current/5/CURRENT_RATING*100):.1f}% of rating")

# Update total system losses
total_22kv_losses = feeder_losses + ring_losses
total_transformer_losses = transformer_losses + main_transformer_losses
total_system_losses = total_22kv_losses + total_132kv_losses + total_transformer_losses

print("\n=== Total System Losses Summary ===")
print(f"22kV Feeder Losses: {feeder_losses:.3f} MW")
print(f"22kV Ring Losses: {ring_losses:.2f} MW")
print(f"132kV System Losses: {total_132kv_losses:.2f} MW")
print(f"Solar Transformer Losses: {transformer_losses:.2f} MW")
print(f"Main Transformer Losses: {main_transformer_losses:.2f} MW")
print(f"Total System Losses: {total_system_losses:.2f} MW")
print(f"System Efficiency: {((total_generation - total_system_losses)/total_generation * 100):.2f}%")


=== Loss Analysis for Complete System ===

Generation Summary:
Generator G1: 100.00 MW
Generator Feeder_1: 21.43 MW
Generator Feeder_2: 21.43 MW
Generator Feeder_3: 21.43 MW
Generator Feeder_4: 21.43 MW
Generator Feeder_5: 21.43 MW
Generator Feeder_6: 21.43 MW
Generator Feeder_7: 21.43 MW
Generator G3: 103.00 MW
Generator GRID_SLACK: 0.00 MW
Total Generation: 353.01 MW

22kV System Losses:

Solar_Line (1.8km, 150MW) (22kV):
Current: 655.27 A
Losses: 0.16 MW
Loss percentage: 0.11%

Hydro Line Hydro_Line_1 (10km):
Current: 222.47 A
Losses: 0.11 MW

Hydro Line Hydro_Line_2 (10km):
Current: 222.47 A
Losses: 0.11 MW

Grid Line Grid_Line_1 (15km):
Current: 769.88 A
Losses: 1.90 MW

Grid Line Grid_Line_2 (15km):
Current: 769.88 A
Losses: 1.90 MW

22/132kV Solar_Transformer_A:
Power Flow: 75.01 MW
Losses: 0.33 MW
Loss percentage: 0.44%
Loading: 75.0%

22/132kV Solar_Transformer_B:
Power Flow: 75.01 MW
Losses: 0.33 MW
Loss percentage: 0.44%
Loading: 75.0%

Main Transformers Analysis (Including

In [5]:
# 22kV Collection System Loss Analysis (Solar at Peak Load)
print("\n=== 22kV Collection System Loss Analysis at Power Plant ===")

# 1. Feeder Cable Parameters (1x500mm² CAS)
FEEDER_LENGTH = 0.5    # km (500 meters within power plant)
R_90 = 0.0786         # Ω/km at 90°C
CURRENT_RATING = 650   # A
feeder_losses = 0
total_feeder_current = 0

# 2. Analyze each feeder
print("\nIndividual Feeder Analysis (500m):")
for i in range(7):
    # Calculate current 
    feeder_power = 21.42  # MW (150MW/7 feeders)
    feeder_current = (feeder_power * 1e6) / (math.sqrt(3) * 22000)  # A
    total_feeder_current += feeder_current
    
    # Calculate losses
    r_total = R_90 * FEEDER_LENGTH  # Total resistance for 500m
    feeder_loss = 3 * (feeder_current ** 2 * r_total) / 1e6  # MW (3-phase)
    feeder_losses += feeder_loss
    
    loading_percentage = (feeder_current / CURRENT_RATING) * 100
    
    print(f"\nFeeder {i+1} (500m):")
    print(f"Power Transfer: {feeder_power:.2f} MW")
    print(f"Current: {feeder_current:.2f} A")
    print(f"Cable Loading: {loading_percentage:.1f}% of rating")
    print(f"Power Loss: {feeder_loss:.3f} MW")
    print(f"Loss Percentage: {(feeder_loss/feeder_power*100):.3f}%")

# 3. Calculate total 22kV collection system statistics
print("\n=== 22kV Collection System Summary ===")
print(f"Number of Feeders: 7")
print(f"Individual Feeder Rating: 21.43 MW")
print(f"Total System Power: 150.00 MW")
print(f"Total Collection System Losses: {feeder_losses:.3f} MW")
print(f"Average Cable Loading: {(total_feeder_current/7/CURRENT_RATING*100):.1f}%")
print(f"Collection System Loss Percentage: {(feeder_losses/150.0*100):.3f}%")

# 4. Cable Specifications
print("\nFeeder Cable Specifications:")
print(f"Cable Type: 1x500mm² CAS")
print(f"Length per feeder: {FEEDER_LENGTH*1000:.0f} meters")
print(f"Resistance at 90°C: {R_90:.4f} Ω/km")
print(f"Current Rating: {CURRENT_RATING} A")

# 5. Theoretical vs Actual Current
theoretical_current = feeder_power * 1e6 / (math.sqrt(3) * 22000)
print("\nCurrent Analysis per Feeder:")
print(f"Theoretical Current at 30MW: {theoretical_current:.1f} A")
print(f"Current Rating Margin: {(CURRENT_RATING - theoretical_current):.1f} A")


=== 22kV Collection System Loss Analysis at Power Plant ===

Individual Feeder Analysis (500m):

Feeder 1 (500m):
Power Transfer: 21.42 MW
Current: 562.13 A
Cable Loading: 86.5% of rating
Power Loss: 0.037 MW
Loss Percentage: 0.174%

Feeder 2 (500m):
Power Transfer: 21.42 MW
Current: 562.13 A
Cable Loading: 86.5% of rating
Power Loss: 0.037 MW
Loss Percentage: 0.174%

Feeder 3 (500m):
Power Transfer: 21.42 MW
Current: 562.13 A
Cable Loading: 86.5% of rating
Power Loss: 0.037 MW
Loss Percentage: 0.174%

Feeder 4 (500m):
Power Transfer: 21.42 MW
Current: 562.13 A
Cable Loading: 86.5% of rating
Power Loss: 0.037 MW
Loss Percentage: 0.174%

Feeder 5 (500m):
Power Transfer: 21.42 MW
Current: 562.13 A
Cable Loading: 86.5% of rating
Power Loss: 0.037 MW
Loss Percentage: 0.174%

Feeder 6 (500m):
Power Transfer: 21.42 MW
Current: 562.13 A
Cable Loading: 86.5% of rating
Power Loss: 0.037 MW
Loss Percentage: 0.174%

Feeder 7 (500m):
Power Transfer: 21.42 MW
Current: 562.13 A
Cable Loading: 86.5%

In [6]:
# Calculate 132kV system losses
line_results = network.get_lines()

# Hydro lines losses (132kV)
hydro_losses = 0
for line_id in ['L1_H1', 'L1_H2']:  # Hydro lines
    line = line_results.loc[line_id]
    hydro_losses += abs(line['p1'] + line['p2'])
    print(f"\nHydro Line {line['name']}:")
    print(f"Current: {line['i1']:.2f} A")
    print(f"Losses: {abs(line['p1'] + line['p2']):.2f} MW")

# Solar plant line losses (132kV)
solar_line_losses = 0
for line_id in ['L2_S1']:  # Solar plant line
    line = line_results.loc[line_id]
    solar_line_losses += abs(line['p1'] + line['p2'])
    print(f"\nSolar Plant Line {line['name']}:")
    print(f"Current: {line['i1']:.2f} A")
    print(f"Losses: {abs(line['p1'] + line['p2']):.2f} MW")

# Grid lines losses (132kV)
grid_losses = 0
for line_id in ['L3_G1', 'L3_G2']:  # Grid lines
    line = line_results.loc[line_id]
    grid_losses += abs(line['p1'] + line['p2'])
    print(f"\nGrid Line {line['name']}:")
    print(f"Current: {line['i1']:.2f} A")
    print(f"Losses: {abs(line['p1'] + line['p2']):.2f} MW")

# Total 132kV losses including all 132kV lines
total_132kv_losses = hydro_losses + solar_line_losses + grid_losses

# Print 132kV system losses breakdown
print("\n=== 132kV System Losses Breakdown ===")
print(f"Hydro Lines Losses: {hydro_losses:.2f} MW")
print(f"Solar Plant Line Losses: {solar_line_losses:.2f} MW")
print(f"Grid Lines Losses: {grid_losses:.2f} MW")
print(f"Total 132kV System Losses: {total_132kv_losses:.2f} MW")


Hydro Line Hydro_Line_1 (10km):
Current: 222.47 A
Losses: 0.11 MW

Hydro Line Hydro_Line_2 (10km):
Current: 222.47 A
Losses: 0.11 MW

Solar Plant Line Solar_Line (1.8km, 150MW):
Current: 655.27 A
Losses: 0.16 MW

Grid Line Grid_Line_1 (15km):
Current: 769.88 A
Losses: 1.90 MW

Grid Line Grid_Line_2 (15km):
Current: 769.88 A
Losses: 1.90 MW

=== 132kV System Losses Breakdown ===
Hydro Lines Losses: 0.21 MW
Solar Plant Line Losses: 0.16 MW
Grid Lines Losses: 3.79 MW
Total 132kV System Losses: 4.17 MW
